In [29]:
import sys
print(sys.executable)


C:\Users\HDGW0570\AppData\Local\Programs\Python\Python312\python.exe


In [30]:
import evidently
print(evidently.__version__)


0.3.2


# Structure du json

In [31]:
import json

with open("data_drift_summary.json", "r") as f:
    data = json.load(f)

list(data.keys())


['version', 'timestamp', 'metrics']

In [32]:
data

{'version': '0.3.2',
 'timestamp': '2025-11-26 22:22:46.817505',
 'metrics': [{'metric': 'DatasetDriftMetric',
   'result': {'drift_share': 0.5,
    'number_of_columns': 10,
    'number_of_drifted_columns': 1,
    'share_of_drifted_columns': 0.1,
    'dataset_drift': False}},
  {'metric': 'DataDriftTable',
   'result': {'number_of_columns': 10,
    'number_of_drifted_columns': 1,
    'share_of_drifted_columns': 0.1,
    'dataset_drift': False,
    'drift_by_columns': {'DAYS_BIRTH': {'column_name': 'DAYS_BIRTH',
      'column_type': 'num',
      'stattest_name': 'Wasserstein distance (normed)',
      'stattest_threshold': 0.1,
      'drift_score': 0.011583291407330004,
      'drift_detected': False,
      'current': {'small_distribution': {'x': [-25195.0,
         -23409.3,
         -21623.6,
         -19837.9,
         -18052.2,
         -16266.5,
         -14480.8,
         -12695.1,
         -10909.4,
         -9123.699999999999,
         -7338.0],
        'y': [2.2885461271774854e-0

In [33]:
# Contenu de metrics
len(data["metrics"])


2

In [34]:
# Contenu de metrics
for i, metric in enumerate(data["metrics"]):
    print(i, metric.get("metric"))


0 DatasetDriftMetric
1 DataDriftTable


In [35]:
# Contient le drift global, le nombre de colonnes, la part de colonnes driftées
data["metrics"][0]


{'metric': 'DatasetDriftMetric',
 'result': {'drift_share': 0.5,
  'number_of_columns': 10,
  'number_of_drifted_columns': 1,
  'share_of_drifted_columns': 0.1,
  'dataset_drift': False}}

# Data Drift

## Drift global

In [36]:
# Drift global
drift_result = data["metrics"][0]["result"]
print("Drift global :", drift_result["dataset_drift"])
print("Colonnes driftées :", drift_result["number_of_drifted_columns"])
print("Part driftée :", drift_result["share_of_drifted_columns"])


Drift global : False
Colonnes driftées : 1
Part driftée : 0.1


In [37]:
# Détail par colonne
data["metrics"][1]

{'metric': 'DataDriftTable',
 'result': {'number_of_columns': 10,
  'number_of_drifted_columns': 1,
  'share_of_drifted_columns': 0.1,
  'dataset_drift': False,
  'drift_by_columns': {'DAYS_BIRTH': {'column_name': 'DAYS_BIRTH',
    'column_type': 'num',
    'stattest_name': 'Wasserstein distance (normed)',
    'stattest_threshold': 0.1,
    'drift_score': 0.011583291407330004,
    'drift_detected': False,
    'current': {'small_distribution': {'x': [-25195.0,
       -23409.3,
       -21623.6,
       -19837.9,
       -18052.2,
       -16266.5,
       -14480.8,
       -12695.1,
       -10909.4,
       -9123.699999999999,
       -7338.0],
      'y': [2.2885461271774854e-05,
       5.165312945677697e-05,
       5.820168012189339e-05,
       6.144148939831923e-05,
       6.633566936909036e-05,
       7.5411730817234e-05,
       7.64686898719546e-05,
       7.079327929552041e-05,
       5.534099746292142e-05,
       2.1472352970355042e-05]}},
    'reference': {'small_distribution': {'x': [-2

## Drift par colonne

In [38]:
import json
import pandas as pd

# Charger le fichier Evidently
with open("data_drift_summary.json", "r") as f:
    data = json.load(f)

# Bloc DataDriftTable = détail par colonne
drift_table = data["metrics"][1]["result"]

# Résumé global (facultatif car aussi dans DatasetDriftMetric)
print("Drift global :", drift_table["dataset_drift"])
print("Colonnes driftées :", drift_table["number_of_drifted_columns"])
print("Part driftée :", drift_table["share_of_drifted_columns"])

# Extraction détaillée
rows = []
for feature, info in drift_table["drift_by_columns"].items():
    rows.append({
        "feature": feature,
        "drift_detected": info["drift_detected"],
        "stattest": info["stattest_name"],
        "drift_score": info["drift_score"],
        "threshold": info.get("stattest_threshold", None),
    })

df_drift = pd.DataFrame(rows)
df_drift


Drift global : False
Colonnes driftées : 1
Part driftée : 0.1


,feature,drift_detected,stattest,drift_score,threshold
0,DAYS_BIRTH,False,Wasserstein distance (normed),0.011583,0.1
1,DAYS_EMPLOYED,False,Wasserstein distance (normed),0.026816,0.1
2,DAYS_ID_PUBLISH,False,Wasserstein distance (normed),0.044393,0.1
3,DAYS_LAST_PHONE_CHANGE,True,Wasserstein distance (normed),0.138977,0.1
4,EXT_SOURCE_1,False,Wasserstein distance (normed),0.032986,0.1
5,EXT_SOURCE_2,False,Wasserstein distance (normed),0.049571,0.1
6,EXT_SOURCE_3,False,Wasserstein distance (normed),0.061799,0.1
7,REGION_RATING_CLIENT,False,Jensen-Shannon distance,0.015309,0.1
8,REGION_RATING_CLIENT_W_CITY,False,Jensen-Shannon distance,0.017247,0.1
9,REG_CITY_NOT_WORK_CITY,False,Jensen-Shannon distance,0.004883,0.1
